<a href="https://colab.research.google.com/github/mhmd2015/AI/blob/main/UIC421Week1NPL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1.1: Load the dataset in a pandas DataFrame; use only **foxnewshealth.txt** and **cnnhealth.txt** for next tasks. Use the tweets column only, for further tasks.

In [36]:
import pandas as pd

def loadDS(file_path):
  try:
    df = pd.read_csv(file_path, sep='|', header=None, names=['tweet_id', 'date', 'tweet'], encoding='latin1',error_bad_lines=False, warn_bad_lines=True)
    print('latin1')
  except UnicodeDecodeError:
    df = pd.read_csv(file_path, sep='|', header=None, names=['tweet_id', 'date', 'tweet'], encoding='ISO-8859-1',error_bad_lines=False, warn_bad_lines=True)
    print('ISO-8859-1')
  except UnicodeDecodeError:
    df = pd.read_csv(file_path, sep='|', header=None, names=['tweet_id', 'date', 'tweet'], encoding='cp1252',error_bad_lines=False, warn_bad_lines=True)
    print('cp1252')
  except UnicodeDecodeError:
    print(file_path+' has error')

  print(len(df))
  return df






In [ ]:
df1 = loadDS('foxnewshealth.txt')
#print(df1.head())
#print(df1['tweet'])

df2 = loadDS('cnnhealth.txt')
#print(df2.head())
#print(df2['tweet'])

df = pd.concat([df1, df2], ignore_index=True)
#print(len(df))
#print(df['tweet'])

Q1.2 : Using regular expressions, extract hashtags in the tweets and store them. Keep a counter for each hashtag.

In [ ]:
import re

# Assuming df is your DataFrame and it has the 'text' column with tweets

# Extract and count hashtags
hashtag_counts = {}
for text in df['tweet']:
    found_hashtags = re.findall(r"#(\w+)", text)
    for hashtag in found_hashtags:
        if hashtag in hashtag_counts:
            hashtag_counts[hashtag] += 1
        else:
            hashtag_counts[hashtag] = 1

# Display the counts
for hashtag, count in hashtag_counts.items():
    print(f"#{hashtag}: {count}")


Q1.3:  Then sentence tokenize each tweet and then word tokenize each sentence. Keep a counter of each word and sort them in descending order. Notice that there are a lot of special characters and repeated words with different cases, like ‘New’ vs ‘new’.


In [45]:
import re

def tokenize_and_count_words_re(texts):
    word_counts = {}

    # Regular expression patterns
    sentence_pattern = r'[.!?]+'
    word_pattern = r'\b\w+\b'

    for text in texts:
        # Sentence tokenization
        sentences = re.split(sentence_pattern, text)
        for sentence in sentences:
            # Word tokenization
            words = re.findall(word_pattern, sentence)
            for word in words:
                cleaned_word = word.lower()  # Lowercase the word for uniformity
                if cleaned_word in word_counts:
                    word_counts[cleaned_word] += 1
                else:
                    word_counts[cleaned_word] = 1

    # Sort the word counts dictionary in descending order of counts
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    return sorted_word_counts


In [46]:
sorted_word_counts = tokenize_and_count_words_re(df['tweet'])

#for word, count in sorted_word_counts:
#    print(f"{word}: {count}")

In [47]:
with open('words_re.txt', 'w') as file:
    for word, count in sorted_word_counts:
        file.write(f"{word}: {count}\n")

Q1.4 Clean the tweets by removing the special characters and unwanted strings like numbers, HTTP links, etc. Use regular expressions to do so. Normalize the case of all the characters in all the tweets.

In [48]:
import re

def clean_tweets(texts):
    cleaned_texts = []

    for text in texts:
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # Remove special characters and numbers
        text = re.sub(r'\W+|\d+', ' ', text)

        # Normalize case
        text = text.lower()

        cleaned_texts.append(text)

    return cleaned_texts

# Example usage:
# Assuming 'df' is your DataFrame with a 'text' column containing tweets
# cleaned_tweets = clean_tweets(df['text'])

# Now 'cleaned_tweets' contains the cleaned and normalized tweets


In [49]:
cleaned_tweets = clean_tweets(df['tweet'])
print(cleaned_tweets)

with open('clean_tweets.txt', 'w') as file:
    for text in cleaned_tweets:
        file.write(f"{text}\n")

['injury prevention programs unpopular with high school coaches ', '  dietary changes to make midlife ', 'massachusetts governor gets head shaved to support charity ', 'dad wins   marathons in   days winnings to help ailing son ', 'possible cure for melanoma ', 'wear orange glasses to get better sleep study says ', 'utah girl   has rare allergy to water ', 'higher purpose in life tied to better brain health ', 'kids may be more likely to use customized playgrounds ', 'marijuana harvested for medical use in chile ', 'museums offer array of programs for the disabled ', 'girl born with heart on wrong side of body prepares for  th birthday ', 'boy gets food allergies from blood transfusion ', 'chinese citizens sue government over transparency on monsanto herbicide ', ' princess lacey tribute begins again for michigan state university campus rock ', 'answering personal questions activates unique brain activity pattern study suggests via mindyc_', 'kailahun ebola case report was a mistake sa

Q1.5 Remove Stopwords from all the text examples. You may use any library like SpaCy or NLTK. Repeat task 3 and notice the difference.

In [50]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [51]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stopwords(texts):
    stop_words = set(stopwords.words('english'))

    cleaned_texts = []
    for text in texts:
        tokens = word_tokenize(text)
        filtered_text = ' '.join([word for word in tokens if word not in stop_words])
        cleaned_texts.append(filtered_text)

    return cleaned_texts

# Example usage:
# Assuming 'cleaned_tweets' contains the cleaned and normalized tweets
tweets_without_stopwords = remove_stopwords(cleaned_tweets)

with open('tweets_without_stopwords.txt', 'w') as file:
    for text in cleaned_tweets:
        file.write(f"{text}\n")


Q1.6 Lemmatize the words in step 5 to their base form using either NLTK or SpaCy. Use stemming to the same words in 5 and observe the difference.

In [52]:
#!pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [53]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

def lemmatize_tweets(tweets):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tweets = []
    for tweet in tweets:
        words = word_tokenize(tweet)
        lemmatized_tweet = ' '.join([lemmatizer.lemmatize(word) for word in words])
        lemmatized_tweets.append(lemmatized_tweet)
    return lemmatized_tweets

def stem_tweets(tweets):
    stemmer = PorterStemmer()
    stemmed_tweets = []
    for tweet in tweets:
        words = word_tokenize(tweet)
        stemmed_tweet = ' '.join([stemmer.stem(word) for word in words])
        stemmed_tweets.append(stemmed_tweet)
    return stemmed_tweets




In [54]:
# Example usage with your tweets data
lemmatized_tweets = lemmatize_tweets(tweets_without_stopwords)
stemmed_tweets = stem_tweets(tweets_without_stopwords)

with open('lemmatized_tweets.txt', 'w') as file:
    for text in lemmatized_tweets:
        file.write(f"{text}\n")

with open('stemmed_tweets.txt', 'w') as file:
    for text in stemmed_tweets:
        file.write(f"{text}\n")

Q1.7 Maintain a corpus of all the words that appeared in the given files.

In [55]:
def create_corpus(tweets):
    corpus = []
    for tweet in tweets:
        words = tweet.split()  # Split each tweet into words
        corpus.extend(words)  # Add the words to the corpus
    return corpus




In [56]:
# Example usage:
# Assuming 'tweets' is a list of tweet texts
corpus = create_corpus(tweets_without_stopwords)
# Now 'corpus' contains all the words from the tweets

with open('corpus.txt', 'w') as file:
    for text in corpus:
        file.write(f"{text}\n")

Q3.1


Implement a method for "Edit Distance Algorithm" , min_edit_dist(target, source, ins_cost, del_cost, sub_cost),
to compute the minimum edit distance between a target word and source word. The
variables ins_cost, del_cost, sub_cost are insertion, deletion, and substitution costs
respectively.

In [59]:
def print_edit_distance(target,source,dist):
    for row in dist:
        print("  ".join(map(str, row)))

def min_edit_dist(target, source, ins_cost=1, del_cost=1, sub_cost=2):

    n = len(target)
    m = len(source)

    # Initialize distance matrix
    dist = [[0 for _ in range(m + 1)] for _ in range(n + 1)]

    # Source prefixes can be transformed into an empty string by deleting all characters
    for i in range(1, n + 1):
        dist[i][0] = i * del_cost

    # Target prefixes can be reached from an empty source prefix by inserting every character
    for j in range(1, m + 1):
        dist[0][j] = j * ins_cost

    for j in range(1, m + 1):
        for i in range(1, n + 1):
            # Check if substitution is required
            if target[i - 1] != source[j - 1]:
                sub = sub_cost
            else:
                sub = 0

            dist[i][j] = min(dist[i - 1][j] + del_cost,       # Deletion
                             dist[i][j - 1] + ins_cost,       # Insertion
                             dist[i - 1][j - 1] + sub)        # Substitution

    print_edit_distance(target,source,dist)
    return dist[n][m]


min_distance = min_edit_dist("purpose", "possible")
print("cost = "+str(min_distance))


0  1  2  3  4  5  6  7  8
1  0  1  2  3  4  5  6  7
2  1  2  3  4  5  6  7  8
3  2  3  4  5  6  7  8  9
4  3  4  5  6  7  8  9  10
5  4  3  4  5  6  7  8  9
6  5  4  3  4  5  6  7  8
7  6  5  4  5  6  7  8  7
cost = 7


Q3.2
Implement a method, spell_checker(target), which will access the corpora generated
in Q1.7 and calculate the minimum edit distance of the target word with all the words in
the corpora. Assume insertion and deletion cost as 1 and substitution cost as 0. Return
the top five words from the above result.**bold

In [67]:
def min_edit_dist(target, source, ins_cost=1, del_cost=1, sub_cost=0):
    n = len(target)
    m = len(source)
    dist = [[0 for _ in range(m + 1)] for _ in range(n + 1)]

    for i in range(1, n + 1):
        dist[i][0] = i * del_cost

    for j in range(1, m + 1):
        dist[0][j] = j * ins_cost

    for j in range(1, m + 1):
        for i in range(1, n + 1):
            if target[i - 1] != source[j - 1]:
                sub = sub_cost
            else:
                sub = 0
            dist[i][j] = min(dist[i - 1][j] + del_cost,
                             dist[i][j - 1] + ins_cost,
                             dist[i - 1][j - 1] + sub)
    return dist[n][m]

def spell_checker(target, corpora):

    # if ins=1, del=1, sub=0
    distances = [(word, min_edit_dist(target, word)) for word in corpora]


    distances.sort(key=lambda x: x[1])
    #distances.sort(key=lambda x: x[1], reverse=True)
    return distances[:5]

def spell_checker_close(target, corpora):

    distances = [(word, min_edit_dist(target, word,1,1,2)) for word in corpora]
    distances.sort(key=lambda x: x[1])
    #distances.sort(key=lambda x: x[1], reverse=True)
    return distances[:5]



will load the corpura from "corpus.txt"
then will call the spell_checker method.
when I test, if the value according to the question : ins=1, del=1, sub=0  then the result will be completely far from the word to search

In [68]:
# Load your corpora from a text file
corpora = []
with open('corpus.txt', 'r') as file:
    corpora = file.read().splitlines()

# Call the spell_checker function according to the question
# ins=1, del=1, sub=0
closest_words = spell_checker("example", corpora)


# Print the closest words
print(closest_words)

[('coaches', 0), ('dietary', 0), ('changes', 0), ('midlife', 0), ('support', 0)]


but when set the values : ins=1, del=1, sub=2 then it will show closely words

In [69]:
# Call the spell_checker_close function according to the value:
# ins=1, del=1, sub=2
closest_words = spell_checker_close("example", corpora)


# Print the closest words
print(closest_words)

[('exam', 3), ('exam', 3), ('exam', 3), ('sample', 3), ('apple', 4)]
